In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import plotly.express as px
from pathlib import Path
import json
import datetime
import pandas as pd
import numpy as np

In [ ]:
from autoguitar.tuning.dataset import get_dataset


df = get_dataset()#.head(500)

In [ ]:
px.scatter(
    df,
    x=df.index,
    y="frequency",
    color="stable",
    hover_data=["steps_derivative", "frequency_derivative"],
)

In [ ]:
px.scatter(
    df,
    x="steps",
    y="frequency",
    color="stable",
    # hover_data=["steps_derivative", "frequency_derivative"],
)

In [ ]:
# log(y) = a * log(x) + b
# y = x^a * exp(b)

In [ ]:
import plotly.graph_objects as go
import plotly.subplots

fig = plotly.subplots.make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df["frequency"],
        mode="lines",
        name="frequency",
    )
)

# add motor events
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df["steps"],
        mode="lines",
        marker=dict(color="red"),
        name="steps",
    ),
    secondary_y=True,
)

fig.update_layout(
    xaxis=dict(title="Datetime"),
    title="Frequency over Time",
)

In [ ]:
from autoguitar.tuning.models import get_linear_regression, naive_linear_regression
from autoguitar.tuning.models import physics_linear_regression

df = get_dataset(loose_steps_max_difference=300)

# model, rmse, predictions_naive = naive_linear_regression(df)
model, rmse_naive, predictions_naive = get_linear_regression(
    df, x_columns=["steps", "loose_steps"], squared_frequency=False
)

model, rmse_physics, predictions_physics = physics_linear_regression(
    df, extra_columns=["loose_steps"]
)

print(rmse_naive, rmse_physics)

In [ ]:
def get_cents_error(y_true: pd.Series, y_pred: pd.Series) -> pd.Series:
    return 1200 * np.log2(y_pred / y_true)

In [ ]:
df["frequency_predicted_naive"] = predictions_naive
df["frequency_predicted_physics"] = predictions_physics
df["error_naive"] = (
    get_cents_error(df["frequency"], df["frequency_predicted_naive"])
    .rolling("1s")
    .mean()
)
df["error_physics"] = (
    get_cents_error(df["frequency"], df["frequency_predicted_physics"])
    .rolling("1s")
    .mean()
)

px.line(
    df,
    x=df.index,
    y=["frequency", "frequency_predicted_naive", "frequency_predicted_physics"],
)

In [ ]:
px.line(
    df.loc[df["stable"]],
    x=df.loc[df["stable"]].index,
    y=["error_naive", "error_physics"],
)